The Notebook explores and showcases Global Wildfire events

In [1]:
import os
import glob
import pandas as pd
import geopandas as gpd
import dask_geopandas as dgpd
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

In [2]:
cluster = LocalCluster()
client = Client(cluster)
print(f"Dask Dashboard: {client.dashboard_link}")

/home/rufaib/miniconda3/envs/pace_chl/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36213 instead
  warnings.warn(


Dask Dashboard: http://127.0.0.1:36213/status


In [ ]:
yearly_fire = {
    "2015": "/workspace/_output/GLOBFIRE_burned_area_full_dataset_2002_2023/original_globfire_filtered_2015.shp", 
    "2016": "/workspace/_output/GLOBFIRE_burned_area_full_dataset_2002_2023/original_globfire_filtered_2016.shp",
    "2017": "/workspace/_output/GLOBFIRE_burned_area_full_dataset_2002_2023/original_globfire_filtered_2017.shp",
    "2018": "/workspace/_output/GLOBFIRE_burned_area_full_dataset_2002_2023/original_globfire_filtered_2018.shp",
    "2019": "/workspace/_output/GLOBFIRE_burned_area_full_dataset_2002_2023/original_globfire_filtered_2019.shp",
    "2020": "/workspace/_output/GLOBFIRE_burned_area_full_dataset_2002_2023/original_globfire_filtered_2020.shp", 
    "2021": "/workspace/_output/GLOBFIRE_burned_area_full_dataset_2002_2023/original_globfire_filtered_2021.shp", 
    "2022": "/workspace/_output/GLOBFIRE_burned_area_full_dataset_2002_2023/original_globfire_filtered_2022.shp", 
    "2023": "/workspace/_output/GLOBFIRE_burned_area_full_dataset_2002_2023/original_globfire_filtered_2023.shp"
}

years = sorted(yearly_fire.keys())

In [ ]:
filtered_gdf = dgpd.read_file(yearly_fire[years[0]], chunksize=2048)

for i in range(1, len(years)):
    current_year = years[i]
    print(f"Processing: {years[i-1]} vs {current_year}")

    # Load the next year
    next_year_gdf = dgpd.read_file(yearly_fire[current_year], chunksize=2048)

    print(f"Checking for intersecting geometeries: {years[i-1]} vs {current_year}")
    intersections = dgpd.sjoin(filtered_gdf, next_year_gdf, how="inner", predicate="intersects")

    def select_unique_intersections(gdf):
        """Finds intersecting indices to remove"""
        gdf = gdf[gdf.index != gdf.index_right]
        return gdf.index
     
    print(f"computing indices of intersecting geometeries: {years[i-1]} vs {current_year}")
    intersecting_ids = intersections.map_partitions(select_unique_intersections).compute()

    print(f"removing intersection of {current_year} from main datasets")
    filtered_gdf = filtered_gdf.map_partitions(lambda df: df[~df.index.isin(intersecting_ids)])

    print(f"Remaining records after filtering {current_year}: {filtered_gdf.compute().shape[0]}")


final_filtered_gdf = filtered_gdf.compute()
final_filtered_gdf
final_filtered_gdf.to_file("final_non_intersecting_wildfires_2015_2023.shp")

print("Final filtered dataset saved!")

Processing: 2015 vs 2016
Remaining records after filtering 2016: 522408
Final filtered dataset saved!


/home/rufaib/miniconda3/envs/pace_chl/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field initialdat create as date field, though DateTime requested.
  ogr_write(
/home/rufaib/miniconda3/envs/pace_chl/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field finaldate create as date field, though DateTime requested.
  ogr_write(
